In [1]:
from aocd import get_puzzle

puzzle = get_puzzle(year=2025, day=10)
data = puzzle.input_data
examples = puzzle.examples

## Part 1

In [3]:
parseddata = []

for l in data.split('\n'):
    lights, *buttons, joltages = l.split()
    lights = lights.strip("[]")
    lights = sum(2**i for i, light in enumerate(lights) if light == "#")

    buttons = [sum(2**int(i) for i in button.strip("()").split(',')) for button in buttons]

    parseddata.append({
        "lights": lights,
        "buttons": buttons
    })

In [ ]:
from queue import PriorityQueue

p1total = 0

for pd in parseddata:
    # print(*pd.values())
    pq = PriorityQueue()
    pq.put((0, pd['lights'], set(pd['buttons'])))
    safe = True

    while pq:
        working = pq.get()

        for button in working[2]:
            newlights = working[1] ^ button
            if newlights == 0:
                p1total += working[0] + 1
                safe = False
                break
            pq.put((working[0] + 1, newlights, working[2] - {button}))
        
        if not safe: break# 

print(p1total)

18 s ± 62.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Optimisations?
Oh. Yeah, okay, PQ was... Too much.

In [ ]:
from operator import xor
from itertools import combinations
from functools import reduce

p1total = 0

for pd in parseddata:
    safe = True
    for x in range(1, len(pd['buttons']) + 1):
        for y in combinations(pd['buttons'], x):
            if pd['lights'] == reduce(xor, y):
                p1total += x
                safe = False
                break
            
        if not safe: break

print(p1total)

11.7 ms ± 922 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)


## Part 2

In [4]:
import numpy as np

parseddata = []

for l in data.split('\n'):
    lights, *buttons, joltages = l.split()

    buttons = [[int(i) for i in button.strip("()").split(',')] for button in buttons]

    joltages = np.array([int(i) for i in joltages.strip('{}').split(',')])

    buttons = [
        np.array([(1 if i in button else 0) for i in range(len(joltages))]) for button in buttons
    ]

    parseddata.append({
        "buttons": sorted(buttons, key=len, reverse=True),
        "joltages": joltages
    })

parseddata = sorted(parseddata, key = lambda pd: max(pd['joltages']))


In [ ]:
from z3 import *
from string import ascii_lowercase

p2total = 0

for pd in parseddata:

    buttonargs = [Int(a) for a, _ in zip(ascii_lowercase, pd['buttons'])]

    conditions = [
        sum(
            b for a, b in enumerate(buttonargs) 
            if pd['buttons'][a][i] == 1
        ) == j for i, j in enumerate(pd['joltages'])
        ]
    constraints = [b >= 0 for b in buttonargs]

    s = Optimize()

    s.add(*conditions, *constraints)

    s.minimize(sum(buttonargs))

    s.check()

    m = s.model()

    p2total += sum(m[d].as_long() for d in m.decls())

print(p2total)